In [ ]:
# Ruta personalizada (cambia esto por tu ruta)
#ruta_guardado = "C:/Users/TuUsuario/Documents/MiCarpeta/salarios.csv"  # Windows
# ruta_guardado = "/home/tuusuario/Documents/MiCarpeta/salarios.csv"  # Linux/macOS

# Guardar el DataFrame en la ruta especificada
#df.to_csv(ruta_guardado, index=False, encoding="utf-8-sig")

#print(f"Archivo guardado en: {ruta_guardado}")

In [ ]:
import re
import numpy as np
import pandas as pd
import ast
from collections import Counter
from rapidfuzz import process, fuzz
from collections import defaultdict

In [ ]:
df = pd.read_csv("../Proyecto final/ofertas_tecnoempleo.csv")

df

In [ ]:
#Código para ver el df completo 
pd.set_option("display.max_columns", None)
df

In [ ]:
# Creamos un id para cada oferta y ponemos la columna en primera posición:

df["id"] = ["id_tec_" + str(i) for i in range(1, len(df) + 1)]
columna_extraida_ = df.pop("id")
df.insert(0, "id", columna_extraida_) 

df

In [ ]:
#Generamos df matricial de tecnologías, lo pasamos a csv y eliminamos las columnas
df_tecnologias_matricial = df[[df.columns[0]] + list(df.columns[25:])]
lista_tecnologias = list(df.columns[25:])

df_tecnologias_matricial.to_csv("tecnologias_matricial_tecnoempleo.csv", index=False, encoding="utf-8-sig")
df.drop(columns=lista_tecnologias, inplace=True)
df_tecnologias_matricial 

In [ ]:
# Renombramos la columna con título vacío (columna con información de trabajo remoto o no):
df.rename(columns={"Unnamed: 7": "Teletrabajo"}, inplace=True)
df["Teletrabajo"]

In [ ]:
# De la información que nos quedamos en la columna de Teletrabajo, unificamos los valores por: 100% en remoto, híbrido y presencial.
df["Teletrabajo"] = df["Teletrabajo"].apply(lambda x: "Remoto" if "Remoto" in x or "Teletrabajo" in x else "Híbrido" if "Híbrido" in x else "Presencial")

# Verificamos los resultados
df["Teletrabajo"].unique()

In [ ]:
# Limpiamos la columna de Fecha de publicación, eliminando el texto que nos sobra y convirtiendo a datetime:
df["Fecha publicacion"] = df["Fecha publicacion"].str.replace("Actualizada", "").str.replace("Nueva", "")
df["Fecha publicacion"] = pd.to_datetime(df["Fecha publicacion"], format="%d/%m/%Y")

df

In [ ]:
# Quitamos los caracteres no deseados de la columna Idiomas y dividimos la columna en dos: una para el idioma y otra para el nivel.
df["Idiomas"] = df["Idiomas"].fillna("")
df["Idiomas"] = df["Idiomas"].str.replace(r"[\t\r\n|]", "", regex=True)

# Extraemos todos los Idiomas y niveles
def extraer_Idiomas_niveles(texto):
    if not texto.strip():  
        return np.nan, np.nan
    matches = re.findall(r"(\w+)\s*\((\w+)\)", texto)
    if not matches:s
        return np.nan, np.nan 
    Idiomas = [match[0] for match in matches]
    niveles = [match[1] for match in matches]

    # Unimos los Idiomas y niveles en cadenas separadas por comas
    return ", ".join(Idiomas), ", ".join(niveles)


df[["Idiomas_limpios", "niveles_limpios"]] = df["Idiomas"].apply(extraer_Idiomas_niveles).apply(pd.Series)


df["Idiomas_limpios"] = df["Idiomas_limpios"].replace("", np.nan)
df["niveles_limpios"] = df["niveles_limpios"].replace("", np.nan)


df.drop(columns=["Idiomas"], inplace=True)

df

In [ ]:
# Dataframe solo con id, idiomas y niveles limpios
df_idiomas1 = df[["id","Idiomas_limpios","niveles_limpios"]]

# Por cada idioma y nivel, creamos un diccionario con el id, idioma y nivel y los añade a una lista
dict_list = []
for i, row in df_idiomas1.iterrows():
    if pd.notna(row["Idiomas_limpios"]) and pd.notna(row["niveles_limpios"]):
        idiomas = row["Idiomas_limpios"].split(", ")
        niveles = row["niveles_limpios"].split(", ")
        for idioma, nivel in zip(idiomas, niveles):
            dict_list.append({"id": row["id"], "idioma": idioma, "nivel": nivel})
    else:
        dict_list.append({"id": row["id"], "idioma": None, "nivel": None})

df.drop(columns = ["Idiomas_limpios", "niveles_limpios"], inplace = True)
# Convertimos la lista de diccionarios a un dataframe
df_idiomas = pd.DataFrame(dict_list)

df_idiomas 

df_idiomas.to_csv('idiomas_tecnoempleo.csv', index=False)

In [ ]:
# Limpiamos la columna de Salario y la unificamos a bruto por año.

# Separamos las columnas para crear una con la unidad reflejada (mes, año, hora):
split_data = df["Salario"].str.split(" Bruto/", expand=True)
df["Rango"] = split_data[0]  
df["Unidad"] = split_data[1] 

# Dividimos los valores para hacer dos columnas de mínimo y máximo:
rango_split = df["Rango"].str.split("-\xa0", expand=True)
df["Min"] = rango_split[0].str.replace("€", "").str.replace(".", "").str.strip().astype(float)
df["Max"] = rango_split[1].str.replace("€", "").str.replace(".", "").str.strip().astype(float)

# Usamos where para cambiar la palabra mes por 12 y hora por 2080 (40h a la semana por 52 semanas):
df["Factor"] = np.where(
    df["Unidad"] == "mes", 12,
    np.where(df["Unidad"] == "hora", 2080, 1)
)
# Multiplicamos las columnas y dividimos por mil para una mejor visualización:
df["salario_desde"] = df["Min"] * df["Factor"] / 1000
df["salario_hasta"] = df["Max"] * df["Factor"] / 1000

df.drop(columns=["Rango", "Unidad", "Min", "Max", "Factor"], inplace=True)

In [ ]:
# Para la información de la columna de habilidades, creamos un nuevo DF:
df_habilidades_te = df[["id", "Habilidades"]].copy()

# Transformamos de string a lista: 
df_habilidades_te["Habilidades"] = df_habilidades_te["Habilidades"].apply(ast.literal_eval)

# Creamos una lista con las habilidades sin duplicar:
todas_habilidades = sum(df_habilidades_te["Habilidades"], [])
habilidades_unicas = list(set(todas_habilidades))

# Creamos columnas binarias para cada habilidad:
for habilidad in habilidades_unicas:
    df_habilidades_te[habilidad] = df_habilidades_te["Habilidades"].apply(lambda x: 1 if habilidad in x else 0)

df_habilidades_te.drop(columns=["Habilidades"], inplace=True)

# Creamos csv matricial de habilidades:
df_habilidades_te.to_csv("habilidades_matricial_tecnoempleo.csv", index=False, encoding="utf-8-sig")

In [ ]:
#Imprescindible residir, cambiamos los valores para unicficarlos:
df["Imprescindible Residir"] = df["Imprescindible Residir"].replace(
    {"España": "País Puesto", "Spain": "País Puesto", "Country": "País Puesto", 
     "Not Required": "No requerido"})
df["Imprescindible Residir"]

In [ ]:
# Vamos a duplicar las filas por ID, para asociar todas las provincias relacionadas con ese ID y hacemos un nuevo DF:
df_provincias = df[["id", "Otras Provincias"]]

df_provincias["Otras Provincias"] = df_provincias["Otras Provincias"].apply(lambda x: str(x).split(", ") if pd.notna(x) else [None])

df_provincias = df_provincias.explode("Otras Provincias").reset_index(drop=True)

df_provincias.to_csv("df_provincias.csv", index=False, encoding="utf-8")

df.drop(columns=["Otras Provincias"], inplace=True)

# Mostrar resultado
print(df_provincias)
# Falta hacer split por la y también

In [ ]:
df.to_csv("limpieza_tecnoempleo.csv", index=False, encoding="utf-8")
